# Optimisation des modèles A et B

Ce carnet montre comment personnaliser la génération synthétique des données, ajuster le bruit et lancer des recherches Optuna pour les étapes d'entraînement **A** et **B**.


## Chargement du module et configuration

Nous exploitons les dataclasses `DataModuleConfig`, `ModelConfig` et `NoiseConfig` pour contrôler la génération des spectres et leurs bruits.


In [ ]:
import importlib.util, pathlib, sys

repo_root = pathlib.Path('..').resolve()
spec = importlib.util.spec_from_file_location('physae', repo_root / 'physae')
physae = importlib.util.module_from_spec(spec)
sys.modules['physae'] = physae
spec.loader.exec_module(physae)

from physae import (
    DataModuleConfig,
    ModelConfig,
    NoiseConfig,
    StageOptimizationConfig,
    optimize_stage_with_optuna,
    build_data_and_model_from_config,
    train_stage_custom,
)


In [ ]:
data_config = DataModuleConfig(
    seed=123,
    n_points=256,
    n_train=2048,
    n_val=512,
    batch_size=32,
    noise_train=NoiseConfig(
        std_add_range=(0.0, 0.005),
        std_mult_range=(0.0, 0.01),
        p_drift=0.2,
        drift_sigma_range=(10.0, 60.0),
        drift_amp_range=(0.002, 0.02),
        p_fringes=0.2,
        fringe_freq_range=(0.5, 6.0),
        fringe_amp_range=(0.001, 0.008),
    ),
    noise_val=NoiseConfig(
        std_add_range=(0.0, 0.002),
        std_mult_range=(0.0, 0.004),
        p_drift=0.0,
        p_fringes=0.0,
        p_spikes=0.0,
    ),
    with_noise_train=True,
    with_noise_val=True,
    freeze_noise_val=True,
)

model_config = ModelConfig(
    lr=1e-4,
    base_lr=1e-4,
    refiner_lr=5e-5,
    predict_params=['sig0', 'dsig', 'mf_CH4', 'P', 'T', 'baseline1', 'baseline2'],
)


### Optimisation de l'étape A


In [ ]:
stage_a_config = StageOptimizationConfig(
    stage='A',
    n_trials=3,
    timeout=600,
    metric_name='val_loss',
    base_stage_kwargs={'epochs': 5, 'enable_progress_bar': False},
)

study_a, metadata_a = optimize_stage_with_optuna(
    stage_a_config,
    data_config=data_config,
    model_config=model_config,
)
print('Meilleur essai (Stage A):', study_a.best_trial.number)
print('Hyperparamètres Stage A:', study_a.best_params)


### Sauvegarde du meilleur modèle A


In [ ]:
best_stage_a_kwargs = physae._default_stage_kwargs('A')
best_stage_a_kwargs.update(stage_a_config.base_stage_kwargs)
best_stage_a_kwargs.update(study_a.best_params)
best_stage_a_kwargs['ckpt_out'] = 'stage_a_best.ckpt'

model_a, train_loader_a, val_loader_a, _ = build_data_and_model_from_config(
    data_config=data_config,
    model_config=model_config,
)
train_stage_custom(model_a, train_loader_a, val_loader_a, **best_stage_a_kwargs)
stage_a_state = {k: v.cpu() for k, v in model_a.state_dict().items()}
print('Métriques Stage A:', getattr(model_a, 'last_stage_metrics', {}))


### Optimisation de l'étape B (B2)


In [ ]:
stage_b_config = StageOptimizationConfig(
    stage='B2',
    n_trials=3,
    timeout=600,
    metric_name='val_loss',
    base_stage_kwargs={'epochs': 5, 'enable_progress_bar': False},
    initial_state_dict=stage_a_state,
)

study_b, metadata_b = optimize_stage_with_optuna(
    stage_b_config,
    data_config=data_config,
    model_config=model_config,
)
print('Meilleur essai (Stage B):', study_b.best_trial.number)
print('Hyperparamètres Stage B:', study_b.best_params)


### Ré-entraînement final avec les meilleurs réglages


In [ ]:
best_stage_b_kwargs = physae._default_stage_kwargs('B2')
best_stage_b_kwargs.update(stage_b_config.base_stage_kwargs)
best_stage_b_kwargs.update(study_b.best_params)

model_b, train_loader_b, val_loader_b, _ = build_data_and_model_from_config(
    data_config=data_config,
    model_config=model_config,
)
model_b.load_state_dict(stage_a_state, strict=False)
train_stage_custom(model_b, train_loader_b, val_loader_b, **best_stage_b_kwargs)
print('Métriques Stage B:', getattr(model_b, 'last_stage_metrics', {}))


## Conclusion

Nous avons défini des plages de génération personnalisées, optimisé les hyperparamètres des étapes A et B avec Optuna et ré-entraîné les modèles sur les meilleurs réglages.
